In [0]:
#******************INIT*******************
!pip install -U torch torchvision numpy opencv-python

# Homework dataset
!wget http://deeplearning.iit.bme.hu/CVS/HW.zip
!unzip -qq HW.zip
!rm HW.zip

# Traffic Sign Classification set
!wget http://deeplearning.iit.bme.hu/CVS/trafficSignsHW.zip
!unzip -qq trafficSignsHW.zip
!rm trafficSignsHW.zip


!git clone https://github.com/szykry/CVML_HW.git

# Set root folder
import os
name = "/content/"
os.chdir(name)
!pwd

 #OpenCV
import cv2

#Numpy - numeric library
import numpy as np

#Plotting
import matplotlib.pyplot as plt
%matplotlib inline

#*******************END OF INIT*************

In [0]:
import torch
import torch.nn as nn
import torchvision
import os
import sys
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import imutils
import cv2

from torch import optim
from torchvision import transforms, datasets, get_image_backend
from torchvision.datasets import ImageFolder
from PIL import Image
from IPython.display import HTML, display

# Class names
classNames = ['traffic sign', 'vehicle', 'cactus']
subclassNames = [
    ['Bump', 'Bumpy road', 'Bus stop', 'Children', 'Crossing (blue)', 'Crossing (red)', 'Cyclists',
     'Danger (other)', 'Dangerous left turn', 'Dangerous right turn', 'Give way', 'Go ahead', 'Go ahead or left',
     'Go ahead or right', 'Go around either way', 'Go around left', 'Go around right', 'Intersection', 'Limit 100',
     'Limit 120', 'Limit 20', 'Limit 30', 'Limit 50', 'Limit 60', 'Limit 70', 'Limit 80', 'Limit 80 over',
     'Limit over', 'Main road', 'Main road over', 'Multiple dangerous turns', 'Narrow road (left)',
     'Narrow road (right)', 'No entry', 'No entry (both directions)', 'No entry (truck)', 'No stopping', 'No takeover',
     'No takeover (truck)', 'No takeover (truck) end', 'No takeover end', 'No waiting', 'One way road',
     'Parking', 'Road works', 'Roundabout', 'Slippery road', 'Stop', 'Traffic light', 'Train crossing',
     'Train crossing (no barrier)', 'Wild animals', 'X - Priority', 'X - Turn left', 'X - Turn right'],
    ['SUV','truck','plane'],
    ['happy','sad','angry','evil']
]
def getTrueSignSubclass(num):
  if num<44:
    return num
  if num==44:
    return 52
  if num<52:
    return num-1
  return num+1
    

class ConvNet(nn.Module):

    def init(self, base_channels=16, in_channels=3, num_classes=55):  # base: csatorna növekedéshez, in: RGB
        super().init()

        # Filters
        self.c11 = Conv(in_channels, base_channels)
        self.c12 = Conv(base_channels, base_channels)
        self.d1  = Conv(base_channels, base_channels2, stride=2) # Downscale using strided convolution and expand channels

        self.c21 = Conv(base_channels2, base_channels2)
        self.c22 = Conv(base_channels2, base_channels2)
        self.d2  = Conv(base_channels2, base_channels4, stride=2)

        self.c31 = Conv(base_channels4, base_channels4)
        self.c32 = Conv(base_channels4, base_channels4)
        self.d3  = Conv(base_channels4, base_channels8, stride=2)

        self.c41 = Conv(base_channels8, base_channels8)
        self.c42 = Conv(base_channels8, base_channels8)
        self.d4  = Conv(base_channels8, base_channels16, stride=2)

        self.c51 = Conv(base_channels16, base_channels16)
        self.c52 = Conv(base_channels16, base_channels16)
        self.d5  = Conv(base_channels16, base_channels32, stride=2) 

        # Input image is 32x32 -> after 5 downscaling the activation map is 1x1
        # [batch, ch, h, w]  -> [batch, ch] linearisba, h=1, w=1
        # Classifier is a normal 1x1 convolution that produces num_classes class scores
        # This layer does not have BatchNorm or ReLU

        self.classifier = nn.Conv2d(base_channels32, num_classes, kernel_size=1)

    def forward(self,x):

        x = self.d1(self.c12(self.c11(x)))
        x = self.d2(self.c22(self.c21(x)))
        x = self.d3(self.c32(self.c31(x)))
        x = self.d4(self.c42(self.c41(x)))
        x = self.d5(self.c52(self.c51(x)))

        return torch.squeeze(self.classifier(x))    # After squeeze is becomes (batch_size x num_classes)

class Conv(nn.Module):

    def init(self, in_channels, channels, k_size=3, stride=1):
        super().init()

        self.conv = nn.Conv2d(in_channels, channels, k_size, stride=stride, padding=k_size//2, bias=False)
        self.bn = nn.BatchNorm2d(channels)
        

    def forward(self,x):
        return self.bn(torch.relu(self.conv(x)))

net=ConvNet()
if torch.cuda.is_available():
  net = net.cuda()
net = torch.load("/content/CVML_HW/model/pyVision_full_tf_2.pth")
net.eval() 

def mergeTheCloseRectanglesR(rectangles,thresh=30):
    #make bigger rect if some rects are close to each other
    #print("Merger")
    for i in range(len(rectangles)):
      for j in range(len(rectangles)):
        if(i>=j):
          continue
        #print(str(i) + str(j)) 
        distance = calculateCenterDistanceR(rectangles[i],rectangles[j])
        #print(distance)
        if(distance < thresh):
          rectangles[i]=makeBigerRectR(rectangles[i],rectangles[j])
          rectangles[j]=rectangles[i]
          #print("At least 2 rects are merged")
    rectangles = deleteSameSizeRectsR(rectangles)
    return rectangles
def calculateCenterDistanceR(rectangle1, rectangle2):
    r1centerx=rectangle1[0]+(rectangle1[2]/2)
    r2centerx=rectangle2[0]+(rectangle2[2]/2)
    r1centery=rectangle1[1]+(rectangle1[3]/2)
    r2centery=rectangle2[1]+(rectangle2[3]/2)
    distance = np.sqrt(abs((r1centerx-r2centerx)**2+(r1centery-r2centery)**2))
    return distance
def makeBigerRectR(first, second):
    result = []

    x = getLower(first[0],second[0])
    xLen= getGreater(first[0]+first[2],second[0]+second[2]) - x

    y = getLower(first[1],second[1])
    yLen= getGreater(first[1]+first[3],second[1]+second[3]) - y

    if(x>y):
        radius = yLen/2
    else:
        radius = xLen/2

    centerX = x+(xLen/2)
    centerY = y+(yLen/2)

    return (x,y,xLen,yLen)

def deleteSameSizeRectsR(rectangles):
    outRectangle=[]
    #print("SameSizeDeleter")
    #delete the same size rects
    for i in range(len(rectangles)):
      isNew=True
      for j in range(len(rectangles)):
        if(i >= j):
          continue
        if(True ==((abs(rectangles[i][0]-rectangles[j][0]) < 10) & (abs(rectangles[i][1]-rectangles[j][1]) < 10) & ((calculateCenterDistanceR(rectangles[i], rectangles[j])) < 20))):
          isNew=False
          #print("Deleted same size rect")
      if(isNew == True):
        outRectangle.append(rectangles[i])
        #print("ADDED")
    return outRectangle


def cropper(image,boundingbox,depth_filter,debug=False):

  image2    = cv2.cvtColor(image.copy(),cv2.COLOR_BGR2RGB)
  image_hsv = cv2.cvtColor(image.copy(),cv2.COLOR_BGR2HSV)
  lower_range = ((1,0,0),(100,0,0),(21,80,0))
  upper_range = ((10,255,255),(120,255,255),(40,255,255))
  #blues = makeMaskWithColorRangesNoMorph(lower_range,upper_range,image_hsv)
  blues = cv2.inRange(image_hsv,lower_range[0],upper_range[0])
  reds  = cv2.inRange(image_hsv,lower_range[1],upper_range[1])
  yellow= cv2.inRange(image_hsv,lower_range[2],upper_range[2])

  imageb=cv2.bitwise_and(image2,image2, mask=blues)
  imager=cv2.bitwise_and(image2,image2, mask=reds)
  imagey=cv2.bitwise_and(image2,image2, mask=yellow)


  #kernel = np.array([[-1,-1,-1], [-1,11,-1], [-1,-1,-1]])
  #image2 = cv2.filter2D(image2, -1, kernel)

  img_gray = cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)

  start = (int(boundingbox[0]),int(boundingbox[1]))
  end   = (int(boundingbox[0]+boundingbox[2]) , int(boundingbox[1]+boundingbox[3]))
  mask  = np.zeros(image.shape,dtype=np.uint8)
  mask  = cv2.rectangle(mask,start, end,(255,255,255),-1)
  mask  = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
  maskwidth  = boundingbox[2]
  maskheight = boundingbox[3]

  blues = cv2.bitwise_and(blues,blues, mask=mask).astype('uint8')
  reds = cv2.bitwise_and(reds,reds, mask=mask).astype('uint8')
  yellow = cv2.bitwise_and(yellow,yellow, mask=mask).astype('uint8')

  blues = cv2.bitwise_and(blues,blues, mask=depth_filter).astype('uint8')
  reds = cv2.bitwise_and(reds,reds, mask=depth_filter).astype('uint8')
  yellow = cv2.bitwise_and(yellow,yellow, mask=depth_filter).astype('uint8')

  kernel  = np.ones((3,3),np.uint8)
  blues = cv2.morphologyEx(blues, cv2.MORPH_CLOSE, kernel,iterations=4)
  reds = cv2.morphologyEx(reds, cv2.MORPH_CLOSE, kernel,iterations=4)
  yellow = cv2.morphologyEx(yellow, cv2.MORPH_CLOSE, kernel,iterations=4)

  if debug:
    plt.figure(figsize=(20,20))
    plt.subplot(1,3,1)
    plt.imshow(blues)
    plt.subplot(1,3,2)
    plt.imshow(reds)
    plt.subplot(1,3,3)
    plt.imshow(yellow)

  sure_fg_b = cv2.erode(blues,kernel,iterations=4)
  sure_bg_b = cv2.dilate(blues,kernel,iterations=2)
  unknown_b = cv2.subtract(sure_bg_b,sure_fg_b)
  sure_fg_r = cv2.erode(reds,kernel,iterations=3)
  sure_bg_r = cv2.dilate(reds,kernel,iterations=2)
  unknown_r = cv2.subtract(sure_bg_r,sure_fg_r)
  sure_fg_y = cv2.erode(yellow,kernel,iterations=2)
  sure_bg_y = cv2.dilate(yellow,kernel,iterations=2)
  unknown_y = cv2.subtract(sure_bg_y,sure_fg_y)

  ret, markers_b = cv2.connectedComponents(sure_fg_b)
  markers_b = markers_b+1
  markers_b[unknown_b==255] = 0
  markers_b = cv2.watershed(image2,markers_b)
  image2[markers_b == -1] = [255,0,0]
  ret, markers_r = cv2.connectedComponents(sure_fg_r)
  markers_r = markers_r+1
  markers_r[unknown_r==255] = 0
  markers_r = cv2.watershed(image2,markers_r)
  image2[markers_r == -1] = [255,0,0]
  ret, markers_y = cv2.connectedComponents(sure_fg_y)
  markers_y = markers_y+1
  markers_y[unknown_y==255] = 0
  markers_y = cv2.watershed(image2,markers_y)
  image2[markers_y == -1] = [255,0,0]

  if debug:
    plt.figure(figsize=(20,20))
    plt.subplot(1,3,1)
    plt.imshow(sure_fg_b,cmap='gray')
    plt.subplot(1,3,2)
    plt.imshow(sure_fg_r,cmap='gray')
    plt.subplot(1,3,3)
    plt.imshow(sure_fg_y,cmap='gray')
  
  rects =[]

  for i in np.unique(markers_b):
    if i==0 | i==-1:
      continue
    tabla = np.where(markers_b == i,255,0).astype('uint8')
    Xmin,Xmax,Ymin,Ymax = minMaxLoc(tabla)
    xCenter = (Xmax+Xmin)/2
    yCenter = (Ymax+Ymin)/2
    w = Xmax-Xmin
    h = Ymax-Ymin
    if (maskwidth<w) | (maskheight<h):
      continue
    if w<32:
      w=32
    if h<32:
      h=32
    x=int(xCenter-w/2)
    y=int(yCenter-h/2)
    if x<0:
      x=0
    if y<0:
      y=0
    rects.append((x,y,w,h))

  for i in np.unique(markers_r):
    if i==0 | i==-1:
      continue
    tabla = np.where(markers_r == i,255,0).astype('uint8')
    Xmin,Xmax,Ymin,Ymax = minMaxLoc(tabla)
    xCenter = (Xmax+Xmin)/2
    yCenter = (Ymax+Ymin)/2
    w = Xmax-Xmin
    h = Ymax-Ymin
    if (maskwidth<w) | (maskheight<h):
      continue
    if w<32:
      w=32
    if h<32:
      h=32
    x=int(xCenter-w/2)
    y=int(yCenter-h/2)
    if x<0:
      x=0
    if y<0:
      y=0
    rects.append((x,y,w,h))

  for i in np.unique(markers_y):
    if i==0 | i==-1:
      continue
    tabla = np.where(markers_y == i,255,0).astype('uint8')
    Xmin,Xmax,Ymin,Ymax = minMaxLoc(tabla)
    xCenter = (Xmax+Xmin)/2
    yCenter = (Ymax+Ymin)/2
    w = Xmax-Xmin
    h = Ymax-Ymin
    if (maskwidth<w) | (maskheight<h):
      continue
    if w<32:
      w=32
    if h<32:
      h=32
    x=int(xCenter-w/2)
    y=int(yCenter-h/2)
    if x<0:
      x=0
    if y<0:
      y=0
    rects.append((x,y,w,h))


  if debug:
    plt.figure(figsize=(20,20))
    plt.subplot(1,3,1)
    plt.imshow(markers_b)
    plt.subplot(1,3,2)
    plt.imshow(markers_r)
    plt.subplot(1,3,3)
    plt.imshow(markers_y)

  if (len(rects)>4):
    truerects= mergeTheCloseRectanglesR(rects,np.floor(maskwidth/4))
  else:
    truerects=rects

  return truerects

'''def prepNeural():
  
  return net'''

def neuralLabeler(img):
  np.moveaxis(img, -1, 0)
  t = transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize([0.485, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])])
  timg = t(img)

  input = torch.zeros(1,3,32,32)
  input[0] = timg

  if torch.cuda.is_available():
    input = input.cuda()
 
  output = net(input)
  _ , pred = torch.max(output, 0)

  return pred

def signFinder(imgBe,dFilter,imgOut,bigsigns,depth,folderName="",picName="",debug=False,output=False):
  boundingboxes=[]
  for j,b in enumerate(bigsigns):
    for i,r in enumerate(cropper(imgBe,b,dFilter,debug=debug)):
      (x,y,w,h) = r
      cv2.rectangle(imgOut,(x,y),(x+w,y+h),(255,0,0),1)
      tabla = imgBe[y:y+h,x:x+w].copy()
      tabla = cv2.resize(tabla,(32,32),interpolation=cv2.INTER_CUBIC)
      tabla = cv2.cvtColor(tabla,cv2.COLOR_BGR2RGB)
      #net = prepNeural()
      signtype=neuralLabeler(tabla)
      #plt.figure(figsize=(5,5))
      #cv2.putText(tabla,subclassNames[0][signtype.cpu().item()],(0,10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(255,0,0))
      #plt.imshow(tabla,cmap='gray')
      u = np.floor(x+w/2)
      v = np.floor(y+h/2)
      x3d,y3d,z3d = get3DCoordinates(u,v, depth)
      boundingboxes.append( np.array([int(u),int(v),w,h ,0 ,getTrueSignSubclass(signtype.cpu().item()), x3d.item(),y3d,z3d],dtype = object)) #subclassNames[0][]
      #cv2.putText(imgOut,subclassNames[0][getTrueSignSubclass(signtype.cpu().item())],(x,y+10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(255,0,0))
      if output:
        cv2.imwrite("/content/HW/pics/"+folderName+"_"+picName+"_"+str(j)+"."+str(i)+"ts.jpg",tabla)

  return boundingboxes

def objFinder(img, val):
    threshold = val
    canny_output = cv2.Canny(img, threshold, threshold * 2)
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])

    outRectangle=[]
    for i in range(len(boundRect)):
      if (int(boundRect[i][2]) > 10) & (int(boundRect[i][3]) > 10):
        outRectangle.append((boundRect[i],centers[i],radius[i]))
    return outRectangle


def makeMaskWithColorRanges(img,low_ranges,high_ranges):
    firstFor = True
    for i in range(len(low_ranges)):
        maszk=cv2.inRange(img,low_ranges[i],high_ranges[i])
        if(firstFor == True):
            result = maszk
            firstFor = False
        else:
          result=cv2.add(result,maszk)
    #plt.figure(figsize=(5,5))
    #plt.imshow(result,cmap='gray')
    kernel = np.ones((5,5),np.uint8)
    result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel, iterations = 2)
    result = cv2.morphologyEx(result, cv2.MORPH_OPEN, kernel, iterations = 1)
    result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, kernel, iterations = 2 )
    #plt.figure(figsize=(5,5))
    #plt.imshow(result,cmap='gray')
    return result

def getGreater(first,second):
    if(first>second):
        return first
    else:
        return second

def getLower(first,second):
    if(first<second):
        return first
    else:
        return second

def makeBigerRect(first, second):
    result = []

    x = getLower(first[0][0],second[0][0])
    xLen= getGreater(first[0][0]+first[0][2],second[0][0]+second[0][2]) - x

    y = getLower(first[0][1],second[0][1])
    yLen= getGreater(first[0][1]+first[0][3],second[0][1]+second[0][3]) - y

    if(x>y):
        radius = yLen/2
    else:
        radius = xLen/2

    centerX = x+(xLen/2)
    centerY = y+(yLen/2)

    return ((x,y,xLen,yLen),(centerX,centerY),(radius))
    
def deleteSameSizeRects(rectangles):
    outRectangle=[]
    #print("SameSizeDeleter")
    #delete the same size rects
    for i in range(len(rectangles)):
      isNew=True
      for j in range(len(rectangles)):
        if(i >= j):
          continue
        if(True ==((abs(rectangles[i][0][0]-rectangles[j][0][0]) < 10) & (abs(rectangles[i][0][1]-rectangles[j][0][1]) < 10) & (abs(rectangles[i][2]-rectangles[j][2]) < 20))):
          isNew=False
          #print("Deleted same size rect")
      if(isNew == True):
        outRectangle.append(rectangles[i])
        #print("ADDED")
    return outRectangle

def calculateCenterDistance(rectangle1, rectangle2):
    p1=rectangle1[1]
    p2=rectangle2[1]
    distance = np.sqrt(abs((p1[0]-p2[0])**2+(p1[1]-p2[1])**2))
    return distance

def mergeTheCloseRectangles(rectangles):
    #make bigger rect if some rects are close to each other
    #print("Merger")
    for i in range(len(rectangles)):
      for j in range(len(rectangles)):
        if(i>=j):
          continue
        #print(str(i) + str(j))   
        distance = calculateCenterDistance(rectangles[i],rectangles[j])
        #print(distance)
        if(distance < 150):
          rectangles[i]=makeBigerRect(rectangles[i],rectangles[j])
          rectangles[j]=rectangles[i]
          #print("At least 2 rects are merged")
    deleteSameSizeRects(rectangles)
    return rectangles

def deleteSmallObjects(rectangles,treshold):
    outRects = []
    for i in range(len(rectangles)):
        if(((rectangles[i][0][2]>treshold) | (rectangles[i][0][3]>treshold)) & (rectangles[i][2]>treshold*0.3)):
            outRects.append(rectangles[i])
        #else:
          #print("Too Small")
    return outRects

def deleteBackground(depth):
    sor =depth[0,:]
    legelsoAtlag=np.true_divide(sor.sum(),np.count_nonzero(sor))
    hatsoMaszk =  np.zeros((480,640)).astype("uint8")
    hatsoMaszk[depth<legelsoAtlag * 0.6]=[1]
    return hatsoMaszk

def makeMaskFromRect(rect):
    rectMaszk =  np.zeros((480,640)).astype("uint8")
    xmin =rect[0][0]-30
    xmax = rect[0][0]+rect[0][2]+30
    ymin = rect[0][1]-30
    ymax = rect[0][1]+rect[0][3]+30

    if(xmin < 0):
        xmin=0
    if(ymin < 0):
        ymin=0
    if(xmax > 639):
        xmax=639
    if(ymax > 479):
        ymax=479

    #print(xmin)
    #print(ymin)
    #print(xmax)
    #print(ymax)
    #print(str(xmin)+" "+str(xmax)+" "+str(ymin)+" "+str(ymax)+" ")
    rectMaszk[ymin:ymax,xmin:xmax]=[1]
    return rectMaszk

def countFilteredValue(image,valueLow, valueHigh):
    tempImage = image.copy()
    lower_range = (0,0,valueLow)
    upper_range = (255,255,valueHigh)
    mask=cv2.inRange(image,lower_range,upper_range)
    return np.count_nonzero(mask)

def isPlane(image,rect):
    tempImage = image.copy()
    rectMask = makeMaskFromRect(rect)

    #print(rect)

    lower_range = (16,100,100)
    upper_range = (25,255,255)
    tempMaszk=cv2.inRange(tempImage,lower_range,upper_range)
    sargaMaszk = cv2.bitwise_and(rectMask,rectMask,mask = tempMaszk)
    sargaDarab = np.count_nonzero(sargaMaszk)
    #print("sarga: "+str(sargaDarab))

    
    lower_range = (0,0,0)
    upper_range = (6,255,255)
    tempMaszk=cv2.inRange(tempImage,lower_range,upper_range)
    rectMask = cv2.bitwise_and(rectMask,rectMask,mask = tempMaszk)
    tempImage = cv2.bitwise_and(tempImage,tempImage,mask = rectMask)
    lowRed = countFilteredValue(tempImage,10,155)
    highRed = countFilteredValue(tempImage,135,255)

    #print(lowRed)
    #print(highRed)
    if((lowRed < highRed) or ((sargaDarab > 15) and ((lowRed/highRed)<1.2)) or ((sargaDarab > 70) and ((lowRed/highRed)<1.4)) or ((sargaDarab > 300))):
        #print("it is a PLANE <3")
        return True
    else:
        return False
    return False

def findRedAndYellowVehicle(imgBGR,depthIn, debug = False):
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
    imgHSV = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2HSV)

    hastsoMaszk = deleteBackground(depthIn)

    

    imgHSV = cv2.bitwise_and(imgHSV,imgHSV,mask = hastsoMaszk)

    if debug:
      plt.figure(figsize=(5,5))
      plt.imshow(imgHSV,cmap='gray')

    lower_range = ((0,195,80),(170,195,80))
    upper_range = ((6,255,255),(180,255,255))
    imgOut = makeMaskWithColorRanges(imgHSV,lower_range,upper_range)

    if debug:
      plt.figure(figsize=(5,5))
      plt.imshow(imgOut,cmap='gray')

    outRedRectangle = objFinder(imgOut,100)



    outRedRectangle = mergeTheCloseRectangles(outRedRectangle)  

    outRedRectangle = deleteSmallObjects(outRedRectangle,30)

    outRedRectangle=deleteSameSizeRects(outRedRectangle)

    if debug:
      print(outRedRectangle)
          
    if debug:
      print("yellow")

    #***YELLOW FINDER***      

    lower_range = ((10,210,200),(10,210,200))
    upper_range = ((30,255,255),(30,255,255))
    imgOut = makeMaskWithColorRanges(imgHSV,lower_range,upper_range)
    if debug:
      plt.figure(figsize=(5,5))
      plt.imshow(imgOut,cmap='gray')

    outYellowRectangle = objFinder(imgOut,100)

    

    outYellowRectangle = mergeTheCloseRectangles(outYellowRectangle)  

    outYellowRectangle = deleteSmallObjects(outYellowRectangle,35)
    outYellowRectangle=deleteSameSizeRects(outYellowRectangle)
    
    if debug:
      print(outYellowRectangle)

    rectStruct = []


    for i in range(len(outYellowRectangle)):
        color = (255,255,0)
        cv2.rectangle(imgRGB, (int(outYellowRectangle[i][0][0]), int(outYellowRectangle[i][0][1])), \
          (int(outYellowRectangle[i][0][0]+outYellowRectangle[i][0][2]), int(outYellowRectangle[i][0][1]+outYellowRectangle[i][0][3])), color, 2)
        
        #cv2.putText(imgRGB,"Truck",(outYellowRectangle[i][0][0],outYellowRectangle[i][0][1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(255,255,0))

        newX = outYellowRectangle[i][0][0]-30
        newY = outYellowRectangle[i][0][1]-30
        newW = outYellowRectangle[i][0][2]+60
        newH = outYellowRectangle[i][0][3]+60

        if(newX < 0):
            xmin=0
        if(newY < 0):
            ymin=0
        if(newX + newW > 639):
            newW = 639-newX
        if(newY + newH > 479):
            newH = 479-newY

        rectStruct.append( ( (int(newX), \
                            int(newY), \
                            int(newW) , \
                            int(newH) )  , 1, 1) )
    

    outRedRectangleFinal = []
    for i in range(len(outRedRectangle)):
        color = (255,0,0)
        itIsTruck = False
        for j in range(len(outYellowRectangle)):
            distance= calculateCenterDistance(outRedRectangle[i],outYellowRectangle[j])
            if(distance < 120):
                itIsTruck = True
                #print("truck part")
        if(itIsTruck == False):
            outRedRectangleFinal.append(outRedRectangle[i])
            cv2.rectangle(imgRGB, (int(outRedRectangle[i][0][0]), int(outRedRectangle[i][0][1])), \
              (int(outRedRectangle[i][0][0]+outRedRectangle[i][0][2]), int(outRedRectangle[i][0][1]+outRedRectangle[i][0][3])), color, 2)
            boolTemp = isPlane(imgHSV,outRedRectangle[i])
            if(boolTemp):
                cv2.putText(imgRGB,"Plane <3",(outRedRectangle[i][0][0],outRedRectangle[i][0][1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(255,0,0))
                rectStruct.append( ( (int(outRedRectangle[i][0][0]),
                                     int(outRedRectangle[i][0][1]) , 
                                     int(outRedRectangle[i][0][2]) , \
                                     int(outRedRectangle[i][0][3])) , 1, 2) )
            else:
                cv2.putText(imgRGB,"Car",(outRedRectangle[i][0][0],outRedRectangle[i][0][1]-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,(255,0,0))
                newX = outRedRectangle[i][0][0]-30
                newY = outRedRectangle[i][0][1]-30
                newW = outRedRectangle[i][0][2]+60
                newH = outRedRectangle[i][0][3]+60

                if(newX < 0):
                    xmin=0
                if(newY < 0):
                    ymin=0
                if(newX + newW > 639):
                    newW = 639-newX
                if(newY + newH > 479):
                    newH = 479-newY

                rectStruct.append( ( (int(newX), \
                                    int(newY), \
                                    int(newW) , \
                                    int(newH) )  , 1, 0) )

    return imgRGB , rectStruct

def minMaxLoc(img):
  Xmin = 999;
  Xmax = 0;
  Ymin = 999;
  Ymax = 0;
  
  for i in range(img.shape[0]):
    sor =img[i,:]
    mean=np.count_nonzero(sor)
    if ((Ymin == 999) & (mean > 0)):
      Ymin=i
    if ( mean > 0):
      Ymax = i

  for i in range(img.shape[1]):
    oszlop =img[:,i]
    mean=np.count_nonzero(oszlop)
    if ((Xmin == 999) & (mean > 0)):
      Xmin=i
    if ( mean > 0):
      Xmax = i

  return Xmin,Xmax,Ymin,Ymax

def watershed(edges,img,debug=False):
  kernel = np.ones((3,3),np.uint8)
  #widen edges to make them touch
  edges = cv2.dilate(edges, kernel,iterations=1)

  ret, thresh = cv2.threshold(edges,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(thresh,cmap='gray')

  sure_fg = cv2.erode(thresh,kernel,iterations=1)
  sure_bg = cv2.dilate(thresh,kernel,iterations=1)

  # Uncertain region(borders)
  unknown = cv2.subtract(sure_bg,sure_fg)

  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(sure_fg,cmap='gray')

  # Label separate foreground pathes
  ret, markers = cv2.connectedComponents(sure_fg)
      
  # Add one to all labels so that sure background is not 0, but 1
  markers = markers+1
  # Now, mark the region of unknown with zero
  markers[unknown==255] = 0

  if debug:
    plt.figure(figsize=(7,7))
    plt.imshow(markers)
    
  # Runwatershed
  markers = cv2.watershed(img,markers)
  if debug:
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.imshow(markers)
    plt.subplot(1,2,2)
    plt.imshow(img)

  return markers

def makeBoundingBox(image):
  imagecopy = image.copy()
  imagecopy = cv2.cvtColor(imagecopy,cv2.COLOR_RGB2GRAY)
  imagecopy[imagecopy != 0] = 255;

  xmin,xmax,ymin,ymax = minMaxLoc(imagecopy)

  rect = (xmin,ymin,xmax-xmin,ymax-ymin)

  #rects = objFinder(imagecopy,100,10,10)

  #for i in range(len(rects)):
  #color = (255,255,0)
  #cv2.rectangle(imagecopy, (rect[0],rect[1]), (rect[0]+rect[2],rect[1]+rect[3]), color, 2)


  #plt.figure(figsize=(20,20))
  #plt.imshow(imagecopy,cmap='gray')
  return rect
  #contours,  = c

#aktualis depsegmenter 23:10
def depth_segmenter(inputImage, inputDepthImage, debug=False):
  # Read images
  img = inputImage.copy()
  depth = inputDepthImage.copy()

  sor =depth[0,:]
  legelsoAtlag=np.true_divide(sor.sum(),np.count_nonzero(sor))

  #hatsoMaszk=cv2.inRange(depth, legelsoAtlag*0.9, np.amax(depth))
  hatsoMaszk =  np.zeros((480,640)).astype("uint8")
  hatsoMaszk[depth<legelsoAtlag*0.6]=[1]

  if debug:
    plt.figure(figsize=(20,20))
    plt.imshow(hatsoMaszk,cmap='gray')

  # convert to RGB
  img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

  lower_range = np.array([90,0,0])
  upper_range = np.array([120,255,220])

  maszk=cv2.inRange(img_hsv, lower_range, upper_range)

  maszkNegative = cv2.bitwise_not(maszk)
  
  if debug:
    out = cv2.bitwise_and(img_hsv,img_hsv,mask = maszkNegative)
    plotolni = cv2.cvtColor(out, cv2.COLOR_HSV2RGB)
    plt.figure(figsize=(20,20))
    plt.imshow(plotolni,cmap='gray')

  hue_channel = img_hsv[:,:,0]
  avg_hue = np.average(hue_channel)
  asdasd, hue_channel_segment = cv2.threshold(hue_channel,avg_hue,255,cv2.THRESH_BINARY)
  kernel = np.ones((5,5),np.uint8)
  maszkDilation = cv2.erode(hue_channel_segment,kernel,iterations = 2)

  maszkDilation = cv2.bitwise_not(maszkDilation)
  maszkNegative = cv2.bitwise_and(maszkDilation,maszkDilation,mask = maszkNegative)
  maszkNegative = cv2.bitwise_and(maszkNegative,maszkNegative,mask = hatsoMaszk)
  
  if debug:
    plt.figure(figsize=(20,20))
    plt.imshow(maszkNegative,cmap='gray')

  # calculate foreground
  for i in range(depth.shape[0]):
    sor =depth[i,:]
    mean=np.true_divide(sor.sum(),np.count_nonzero(sor))
    sor[sor>mean-20]=0

  # for some reason doesn't work without this
 # depth2[depth==0]=0
 # img_rgb = np.where(depth2 != 0, img_rgb, [0,0,0])

  # Figure with subplots
  if debug:
    plt.figure(figsize=(30,30))
    plt.subplot(1,2,1)
    plt.imshow(img_rgb) 
    plt.subplot(1,2,2)
    plt.imshow(depth,cmap='gray')

  # convert to uint8 from uint16
  cv2.normalize(depth, depth, 0, 255, cv2.NORM_MINMAX)
  depth=depth.astype('uint8')
  depth_8 = cv2.cvtColor(depth, cv2.COLOR_GRAY2BGR)

  if debug:
    plt.figure(figsize=(30,30))
    plt.subplot(1,2,1)
    plt.imshow(depth_8) 

  kernel = np.ones((3,3),np.uint8)
  depth_8 = cv2.morphologyEx(depth_8, cv2.MORPH_OPEN,  kernel, iterations = 3)
  depth_8 = cv2.morphologyEx(depth_8, cv2.MORPH_CLOSE, kernel, iterations = 3)

  Atlag=np.true_divide(sor.sum(),np.count_nonzero(sor))
  depth_8[depth_8 > Atlag*2.2]=[0]

  depth_out = depth_8
  depth_out = cv2.cvtColor(depth_out,cv2.COLOR_BGR2GRAY)
  depth_out[depth_out>0]=1

  depth_8 = cv2.bitwise_and(depth_8,depth_8,mask = maszkNegative)
  depth_8 = cv2.morphologyEx(depth_8, cv2.MORPH_CLOSE, kernel, iterations = 3)

  if debug: 
    plt.subplot(1,2,2)
    plt.imshow(depth_8,cmap='gray')

  # get edges
  edges = cv2.Canny(depth_8,50,120)
  markers = watershed(edges,img,debug=debug)

  ####
  img2=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  boundingRects=[]
  for i in np.unique(markers):
    if i<2 :
      continue

    imgcopy=img.copy()
    imgcopy[markers!=i]=[0,0,0]  
    xmin,xmax,ymin,ymax = minMaxLoc(imgcopy)

    if ((xmax-xmin > 500) & (ymax-ymin > 350)):
      continue
    
    imgcopy = cv2.bitwise_and(imgcopy,imgcopy,mask = maszkNegative)

    if np.count_nonzero(imgcopy)<3600:
      continue

    #if np.var(np.nonzero(cv2.cvtColor(imgcopy,cv2.COLOR_RGB2HSV)[:,:,0]))<4200:
    #  continue
    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(imgcopy)
      print(str(i)+":"+str(np.var(np.nonzero(cv2.cvtColor(imgcopy,cv2.COLOR_RGB2HSV)[:,:,0])))+"........\t"+str(np.count_nonzero(imgcopy))+"minmax:"+str(xmin)+" "+str(xmax)+" "+str(ymin)+" "+str(ymax))
  
    boundingRects.append(makeBoundingBox(imgcopy))

  for i in range(len(boundingRects)):
    color = (255,0,0)
    start = (int(boundingRects[i][0]),int(boundingRects[i][1]))
    end = (int(boundingRects[i][0]+boundingRects[i][2]) , int(boundingRects[i][1]+boundingRects[i][3]))
    img2 = cv2.rectangle(img2, start, end, (255,0,0), 2)

  return img2, boundingRects, depth_out

def drawRects(image,rects,color):
    outImg = image.copy()
    for i in range(len(rects)):
        start = (int(rects[i][0]),int(rects[i][1]))
        end = (int(rects[i][0]+rects[i][2]) , int(rects[i][1]+rects[i][3]))
        outImg = cv2.rectangle(outImg, start, end, color, 2)
        #print(start,end)
    return outImg

def makeGaborStyleRects(unalmasRect):
    return (unalmasRect,(unalmasRect[0]+unalmasRect[2]/2,unalmasRect[1]+unalmasRect[3]/2))

def makeMaskFromRect2(rect):
    rectMaszk =  np.zeros((480,640)).astype("uint8")
    xmin =rect[0][0]
    xmax = rect[0][0]+rect[0][2]
    ymin = rect[0][1]
    ymax = rect[0][1]+rect[0][3]

    rectMaszk[ymin:ymax,xmin:xmax]=[1]
    return rectMaszk

def deleteVehicleFromDephsegmenter(vehicleRects,depsegmeterRects,tresh):
    outRects = []
    for depIterator in range(len(depsegmeterRects)):
        isVehicle = False
        for vehicleIterator in range(len(vehicleRects)):
            dist = calculateCenterDistance(makeGaborStyleRects(vehicleRects[vehicleIterator]),makeGaborStyleRects(depsegmeterRects[depIterator]))

            if(dist < tresh):
                #print(vehicleRects[vehicleIterator],depsegmeterRects[depIterator])
                #print (dist)
                #print("deleted double vehicle")
                isVehicle = True
        if isVehicle==False: 
            outRects.append(depsegmeterRects[depIterator])
    return outRects

def deleteTableElements(image, debug = False):
    #showSpektrumMask(image, False, 0, 0, 10, 20)
    #image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
    kernel = np.ones((3,3),np.uint8)
    lower_range = (0,0,1)
    upper_range = (3,255,255)
    tempMaszk=cv2.inRange(image,lower_range,upper_range)
    #plt.figure(figsize=(10,10))
    #plt.imshow(tempMaszk,cmap='gray')
    tempMaszk = cv2.morphologyEx(tempMaszk, cv2.MORPH_CLOSE, kernel, iterations = 6)
    tempMaszk = cv2.bitwise_not(tempMaszk)
    if debug: 
      plt.figure(figsize=(10,10))
      plt.imshow(tempMaszk,cmap='gray')
    #dilation = cv2.morphologyEx(ures, cv2.MORPH_OPEN, kernel, iterations = 1)
    image = cv2.bitwise_and(image,image,mask = tempMaszk)

    lower_range = (170,1,0)
    upper_range = (180,255,255)
    tempMaszk=cv2.inRange(image,lower_range,upper_range)
    #plt.figure(figsize=(10,10))
    #plt.imshow(tempMaszk,cmap='gray')
    tempMaszk = cv2.bitwise_not(tempMaszk)
    image = cv2.bitwise_and(image,image,mask = tempMaszk)

    lower_range = (90,1,0)
    upper_range = (110,255,255)
    tempMaszk=cv2.inRange(image,lower_range,upper_range)
    #plt.figure(figsize=(10,10))
    #plt.imshow(tempMaszk,cmap='gray')
    tempMaszk = cv2.bitwise_not(tempMaszk)
    image = cv2.bitwise_and(image,image,mask = tempMaszk)

    #plt.figure(figsize=(10,10))
    #plt.imshow(image,cmap='gray')

    return image

def isCacti(maskedImage, tresh, debug = False):
    tempImage = maskedImage.copy()

    tempImage2 = cv2.cvtColor(tempImage, cv2.COLOR_HSV2RGB)
    if debug: 
      plt.figure(figsize=(10,10))
      plt.imshow(tempImage2,cmap='gray')

    vizsgalhato = deleteTableElements(tempImage, debug)

    vizsgalhatoRGB = cv2.cvtColor(vizsgalhato, cv2.COLOR_HSV2RGB)

    if debug: 
      plt.figure(figsize=(10,10))
      plt.imshow(vizsgalhatoRGB,cmap='gray')

    lower_range = (1,0,1)
    upper_range = (255,100,255)
    tempMaszk=cv2.inRange(vizsgalhato,lower_range,upper_range)
    tableMaszk = cv2.bitwise_and(vizsgalhato,vizsgalhato,mask = tempMaszk)

    kernel = np.ones((5,5),np.uint8)
 #   tempMaszk = cv2.morphologyEx(tempMaszk, cv2.MORPH_CLOSE, kernel, iterations = 2)

    tableDarab = np.count_nonzero(tempMaszk)

    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(tempMaszk,cmap='gray')

    lower_range = (1,100,20)
    upper_range = (255,200,255)
    tempImage = maskedImage.copy()
    tempMaszk=cv2.inRange(vizsgalhato,lower_range,upper_range)
    cactiMaszk = cv2.bitwise_and(vizsgalhato,vizsgalhato,mask = tempMaszk)
   # tempMaszk = cv2.morphologyEx(tempMaszk, cv2.MORPH_CLOSE, kernel, iterations = 2)
    cactiDarab = np.count_nonzero(tempMaszk)

    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(tempMaszk,cmap='gray')

    if debug:
      print("cactiDB: " + str(cactiDarab))
      print("tableDB: " + str(tableDarab))
    if(cactiDarab > tableDarab* 0.4):
        return True
    else:
        return False

def isBenne(bigger, inside):
  if (( (bigger[0] < inside[0]) and (bigger[0]+bigger[2] > inside[0] + inside[2]) ) and ( (bigger[1] < inside[1]) and (bigger[1]+bigger[3] > inside[1] + inside[3]) ) and (inside[2] <70) and (inside[3] < 70)):
      #print("benne")
      return True
  else:
      return False

def deleteVehicleFromTableAndCacti(vehicle, table, cacti):
  outRects = []
  for vIterator in range(len(vehicle)):
      isOk = True
      for tIterator in range(len(table)):
          if isBenne(table[tIterator],vehicle[vIterator][0]):
              #print("tablaban")
              isOk = False

      for cIterator in range(len(cacti)):
          if isBenne(cacti[cIterator],vehicle[vIterator][0]):
              isOk = False
      if isOk:
          outRects.append(vehicle[vIterator])
  return outRects

def cactiAndTableFinder(rects,maszk,image, debug = False):
    tempImage = image.copy()


    tempImage = cv2.cvtColor(tempImage, cv2.COLOR_BGR2HSV)
    cacti = []
    table = []
    maszk[maszk>0]=1
    hattermaszkolt = cv2.bitwise_and(tempImage,tempImage,mask = maszk)

    for i in range(len(rects)):
        rectMask = makeMaskFromRect2((rects[i],""))
        maszkolt = cv2.bitwise_and(hattermaszkolt,hattermaszkolt,mask = rectMask)
        tempBool = isCacti(maszkolt,rects[i][2]*rects[i][3]*0.20 ,debug)
        if tempBool == True:
            cacti.append(rects[i])
        else: 
            table.append(rects[i])
    return cacti,table

def makeTableRect(nagytabla):
  kistabla=[]
  for t in nagytabla:
    if t[3]>t[2]:
      tlist=list(t)
      tlist[3]=tlist[2]
      kistabla.append(tuple(tlist))
    else:
      kistabla.append(t)
  return kistabla



##ORSI stuff


def findMax2(matches):
  maximum = np.sqrt(matches[0][0]**2+matches[0][1]**2)
  i = 0
  index = 0
  for i in range(len(matches)):
    if(np.sqrt(matches[i][0]**2+matches[i][1]**2) >= maximum):
      maximum = np.sqrt(matches[i][0]**2+matches[i][1]**2)
      index = i
  
  return index

def findMaxBoundingRectsCacti(boundingRects):
  maxW = boundingRects[0][2]
  maxH = boundingRects[0][3]
  i = 0
  index = 0
  for i in range(len(boundingRects)):
    if(boundingRects[i][2] > maxW or boundingRects[i][3] > maxH):
      maxW = boundingRects[i][2]
      maxH = boundingRects[i][3]
      index = i
  
  return index

def findMaxBoundingRects(boundingRects):
  maxX = boundingRects[0][2]
  maxY = boundingRects[0][3]
  i = 0
  index = 0
  for i in range(len(boundingRects)):
    if(boundingRects[i][2] > maxX or boundingRects[i][3] > maxY):
      maxX = boundingRects[i][2]
      maxY = boundingRects[i][3]
  
  return (maxX,maxY)

def createTemplate(filePathes):
  ths = []
  tws = []
  temptemplates = []
  templates = []
  for filepath in filePathes:
    template = cv2.imread(filepath)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    template = cv2.Canny(template, 50, 250)
    temptemplates.append(template)
    tws.append(template.shape[1])
    ths.append(template.shape[0])  
  return temptemplates

def objFinder2(img, val):
    threshold = val
    canny_output = cv2.Canny(img, threshold, threshold * 2)
    contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])

    outRectangle=[]
    for i in range(len(boundRect)):
      if (int(boundRect[i][2]) > 10) & (int(boundRect[i][3]) > 10):
        outRectangle.append(boundRect[i])
    return outRectangle

def drawRects2(image,rects,color):
    outImg = image.copy()
    for i in range(len(rects)):
        start = (int(rects[i][0]),int(rects[i][1]))
        end = (int(rects[i][0]+rects[i][2]) , int(rects[i][1]+rects[i][3]))
        outImg = cv2.rectangle(outImg, start, end, color, 2)
        #print(start,end)
    return outImg

def filterOutCacti(objects, masks, boundingRects):
  cacti = []
  objectplaces = []
  avgSs = []
  for i in range(len(objects)):
    maszk = masks[i]
    #maszk = cv2.cvtColor(masks[i],cv2.COLOR_BGR2GRAY)
    valamiS = cv2.cvtColor(objects[i],cv2.COLOR_BGR2HSV)
    valamiS = cv2.bitwise_and(valamiS,valamiS,mask = maszk)
    #showSpektrumMask(valamiS,False,1,0,10,26)
    lower_range = (0,0,1)
    upper_range = (255,70,255)
    tempMaszk=cv2.inRange(valamiS,lower_range,upper_range)
    valamiS = cv2.bitwise_and(valamiS,valamiS,mask = tempMaszk)
    backgroundout = valamiS
    valamiS = cv2.morphologyEx(valamiS, cv2.MORPH_OPEN, (3,3), iterations = 1)
    valamiS = cv2.morphologyEx(valamiS, cv2.MORPH_CLOSE, (3,3), iterations = 2)
    boundingBoxes = objFinder2(valamiS, 100)
    if boundingBoxes == []:
      blank_image = np.zeros((objects[i].shape[0],objects[i].shape[1],3), np.uint8)
      cacti.append(blank_image)
      continue
    index = findMaxBoundingRectsCacti(boundingBoxes)
    imgkep = drawRects2(valamiS, boundingBoxes, (0,255,0))
    blank_image = np.zeros((objects[i].shape[0],objects[i].shape[1],3), np.uint8)
    blank_image[boundingBoxes[index][1]:boundingBoxes[index][1]+boundingBoxes[index][3],boundingBoxes[index][0]:boundingBoxes[index][0]+boundingBoxes[index][2]] = (255,255,255)
    blank_image = cv2.copyTo(objects[i],blank_image)
    blank_image = cv2.cvtColor(blank_image,cv2.COLOR_HSV2BGR)
    cacti.append(blank_image)

    xlength = boundingRects[i][1]+boundingRects[i][3]
    ystart = boundingRects[i][0]
    objectplaces.append((ystart, xlength))

  return cacti, objectplaces

def getCactiSubClass2(ret, boundingRects, depth_out, debug = False):
  #get the copped objects
  objects = []
  masks = []
  templates = []
  returnVal = []
  for i in range(len(boundingRects)):
    img2 = ret[int(boundingRects[i][1]):int(boundingRects[i][1]+boundingRects[i][3]), int(boundingRects[i][0]):int(boundingRects[i][0]+boundingRects[i][2])]
    imgMask = depth_out[int(boundingRects[i][1]):int(boundingRects[i][1]+boundingRects[i][3]), int(boundingRects[i][0]):int(boundingRects[i][0]+boundingRects[i][2])]
    objects.append(img2)
    masks.append(imgMask)
  maxX, maxY = findMaxBoundingRects(boundingRects)
  #resize objects to same size
  for i in range(len(objects)):
    y = objects[i].shape[0]
    x = objects[i].shape[1]
    yratio = maxY / y
    xratio = maxX / x
    final = max(yratio, xratio)
    objects[i] = cv2.resize(objects[i], (int(x*final), int(y*final)))
    masks[i] = cv2.resize(masks[i], (int(x*final), int(y*final)))

  #create templates
  filePathes = ['/content/CVML_HW/data/eyebrowface.png','/content/CVML_HW/data/eyeface.png', '/content/CVML_HW/data/sadfaceface.png', '/content/CVML_HW/data/smileface.png' ]
  templates = createTemplate(filePathes)

  if debug:
    for temp in templates:
      plt.figure(figsize=(10,10))
      plt.imshow(temp)

  tH = templates[0].shape[0]
  tW = templates[0].shape[1]
  objects, objectplaces = filterOutCacti(objects, masks, boundingRects)

  i = -1
  for obj in objects:
    if(np.count_nonzero(obj)==0):
      returnVal.append(-1)
    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(obj)
    i += 1
    matches = []
    realmatches = []
    text = "default text"
    
    obj = cv2.cvtColor(obj, cv2.COLOR_HSV2BGR)
    obj = cv2.bilateralFilter(obj, 150, 50, 50)
    gray = cv2.cvtColor(obj, cv2.COLOR_BGR2GRAY)
    if gray.shape[0] < tH or gray.shape[1] < tW:
      returnVal.append(-1)
      continue
    #minVal, maxVal = cv2.min
    #gray = cv2.convertScaleAbs(gray, gray, 255/(maxVal-minVal), -minVal)
    #_,gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(gray, cmap = 'gray')
    v = np.median(gray)
    sigma = 0.33
    lower_tresh = int(max(0, (1.0-sigma)*v))
    upper_tresh = int(min(255, (1.0+sigma)*v))
    lower_tresh = 70
    upper_tresh = 200
    edged = cv2.Canny(gray, lower_tresh, upper_tresh)
    if debug:
      plt.figure(figsize=(10,10))
      plt.imshow(edged)

    result = cv2.matchTemplate(edged, templates[0], cv2.TM_CCORR_NORMED)
    (_, maxVal0, _, maxLoc0) = cv2.minMaxLoc(result)
    match = (maxVal0, maxLoc0, 0)
    matches.append(match)
    result = cv2.matchTemplate(edged, templates[1], cv2.TM_CCORR_NORMED)
    (_, maxVal1, _, maxLoc1) = cv2.minMaxLoc(result)
    #match = (maxVal1, maxLoc1, r, 1)
    #matches.append(match)

    if maxVal0 > maxVal1 : #has eyebrow
      result = cv2.matchTemplate(edged, templates[2], cv2.TM_CCORR_NORMED)
      (_, maxVal2, _, maxLoc2) = cv2.minMaxLoc(result)
      match = (maxVal2, maxLoc2, 2)
      matches.append(match)
      result = cv2.matchTemplate(edged, templates[3], cv2.TM_CCORR_NORMED)
      (_, maxVal3, _, maxLoc3) = cv2.minMaxLoc(result)
      match = (maxVal3, maxLoc3, 3)
      matches.append(match)

      if maxVal2 > maxVal3: #sad
        text = "angry cacti"
        returnVal.append(2)
        realmatches.append((maxVal0, maxVal2, "angry cactus"))
      else: #smile
        text = "evil cacti"
        returnVal.append(3)
        realmatches.append((maxVal0, maxVal3, "evil cactus"))

    else: #does not have eyebrow
      result = cv2.matchTemplate(edged, templates[2], cv2.TM_CCORR_NORMED)
      (_, maxVal2, _, maxLoc2) = cv2.minMaxLoc(result)
      match = (maxVal2, maxLoc2, 2)
      matches.append(match)
      result = cv2.matchTemplate(edged, templates[3], cv2.TM_CCORR_NORMED)
      (_, maxVal3, _, maxLoc3) = cv2.minMaxLoc(result)
      match = (maxVal3, maxLoc3, 3)
      matches.append(match)

      if maxVal2 > maxVal3: #sad
        text = "sad cacti"
        returnVal.append(1)
        realmatches.append((maxVal1, maxVal2, "sad cactus"))
      else: #smile
        text = "happy cacti"
        returnVal.append(0)
        realmatches.append((maxVal1, maxVal3, "happy cactus"))

    if matches == []:
      continue

    index = findMax2(realmatches)
    _,_,text = realmatches[index]   
    #ret = cv2.putText(ret, text, ( boundingRects[i][0],boundingRects[i][1]+boundingRects[i][3] ), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_4)
  return returnVal

#u - bounding box center x, v - bounding box center y, filePath - given image depth filePath
def get3DCoordinates(u,v, depth):
  img = depth
  z = img[int(v),int(u)]
  x = (u*z-314.4262695300000*z)/606.0482177699999
  y = -1*(v*z-246.0503845200000*z)/604.9857788
  return x/1000,y/1000,z/1000

##MAIN
def processImage(img, depth):

  retRGB, VrecStruct = findRedAndYellowVehicle(img,depth) #TODO  2 64-re szar eredmenyt hoz, pedig az ereditiben jo

  outVrect = []
  for i in range(len(VrecStruct)):
      outVrect.append(VrecStruct[i][0])
      #print(VrecStruct[i][0])

  imgOut, Drects, outDepth = depth_segmenter(img,depth,debug=False)

  kaktusAndTabla = deleteVehicleFromDephsegmenter(outVrect,Drects,60)

  #plt.imshow(drawRects(retRGB,kaktusAndTabla,(0,0,255)),cmap='gray')

  cacti, table = cactiAndTableFinder(kaktusAndTabla,outDepth,img,debug=False)

  table = makeTableRect(table)

  VrecStruct = deleteVehicleFromTableAndCacti(VrecStruct, table,cacti)
      
  #print("cacti: " + str(cacti)+"table:" + str(table))

  resultBoundingBoxes = signFinder(img,outDepth,retRGB,table,depth)
  subClassIndexes = getCactiSubClass2(img, cacti, outDepth)

  for i in range(len(cacti)):
    subclassIndex = subClassIndexes[i]
    if subclassIndex == -1:
      retRGB = cv2.putText(retRGB, "fos", ( cacti[i][0],cacti[i][1]+cacti[i][3] ), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_4)
      continue
    u=np.floor(cacti[i][0]+cacti[i][2]/2)
    v=np.floor(cacti[i][1]+cacti[i][3]/2)
    x,y,z = get3DCoordinates(u,v, depth)
    resultBoundingBoxes.append( np.array([int(u),int(v),cacti[i][2],cacti[i][3], 2, subclassIndex, x,y,z],dtype = object) ) 
  
  for veh in VrecStruct:
    u=np.floor(veh[0][0]+veh[0][2]/2)
    v=np.floor(veh[0][1]+veh[0][3]/2)
    x,y,z = get3DCoordinates(u,v, depth)
    resultBoundingBoxes.append( np.array([int(u),int(v),veh[0][2],veh[0][3], veh[1], veh[2], x,y,z],dtype = object) )


  retRGB = drawRects(retRGB,cacti,(0,255,0))
  retRGB = drawRects(retRGB,table,(255,255,255))

  return retRGB,resultBoundingBoxes


##ITERATE

# egyesitettttt **************** juhhhuuuuu ****** dunnnyezóóóóóó
'''
img = cv2.imread("./HW/g2/rgb/236.jpg")
depth = cv2.imread("./HW/g2/depth/236.png",-1)

retRGB,resultBoundingBoxes = processImage(img,depth)

plt.figure(figsize=(10,10))
plt.imshow(retRGB,cmap='gray')
'''
colors = [(0,0,255),(255,0,255),(0,255,0)]

def drawBBs(BBs, img):
    img = cv2.resize(img, (1280, 960))
    for BB in BBs:
        u = BB[0]*2
        v = BB[1]*2
        w = BB[2]*2
        h = BB[3]*2
        c = BB[4]
        sc = BB[5]
        x = BB[6]
        y = BB[7]
        z = BB[8]
        s = (u - w // 2, v - h // 2)
        e = (u + w // 2, v + h // 2)
        cv2.rectangle(img, s, e, colors[c], 1)
        tl = (s[0], s[1]+15)
        bl = (s[0], e[1]-5)
        cv2.putText(img,subclassNames[c][sc],tl,cv2.FONT_HERSHEY_COMPLEX_SMALL,0.75,colors[c])
        coords = "(%.2f, %.2f, %.2f)" % (x,y,z)
        cv2.putText(img,coords,bl,cv2.FONT_HERSHEY_COMPLEX_SMALL,0.65,colors[c])
    
    return img
'''
import pickle
import cv2
import matplotlib.pyplot as plt
#This way it doesn't try to open a window un the GUI - works in python notebook
%matplotlib inline


# Visualization
depth = depth / 5000.0
img = drawBBs(resultBoundingBoxes, img)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

# Figure with subplots
plt.figure(figsize=(30,30))
plt.subplot(1,2,1)
plt.imshow(img_rgb)
plt.subplot(1,2,2)
plt.imshow(depth,cmap='gray')
'''

import pickle

if __name__ == '__main__':

  poses = np.zeros((12), dtype=int)

  myAnswers={}
  import glob
  foldernames=["g1","g2","g3","g4"]
  for folder in foldernames:
    print(folder)
    for file in glob.glob("./HW/"+folder+"/rgb/*.*"):
      img = cv2.imread(file)
      asd = file.split("/rgb/")
      asd.append(asd[1].split(".jpg"))
      depth = cv2.imread(asd[0]+"/depth/"+asd[2][0]+".png",-1)
      retRGB,resultBoundingBoxes = processImage(img,depth)
      # retRGB BGR ? 
      #img_to_save = drawBBs(resultBoundingBoxes, retRGB)
      #img_to_save = cv2.cvtColor(img_to_save,cv2.COLOR_BGR2RGB)
      #cv2.imwrite("/content/all_imagess/" + folder + "_" + asd[2][0] + ".jpg", img_to_save)

      myPred = {
          'poses' : poses,
          'objects' : resultBoundingBoxes
      }
      key = "HW/"+folder+"/rgb/"+asd[2][0]+".jpg"
      newItem={
          key:myPred
      }
      myAnswers.update(newItem)



In [0]:
import cv2
import numpy as np
import pickle
import os
import os.path as osp

def rbf(val,sigma=0.1):
    return np.exp(-val/sigma)

def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes
    """

    # Transform from center and width to exact coordinates
    b1_x1, b1_x2 = box1[0] - box1[2] / 2, box1[0] + box1[2] / 2
    b1_y1, b1_y2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
    b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
    b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = np.maximum(b1_x1, b2_x1)
    inter_rect_y1 = np.maximum(b1_y1, b2_y1)
    inter_rect_x2 = np.minimum(b1_x2, b2_x2)
    inter_rect_y2 = np.minimum(b1_y2, b2_y2)
    # Intersection area
    inter_area1 = np.maximum(inter_rect_x2 - inter_rect_x1 + 1, 0)
    inter_area2 = np.maximum(inter_rect_y2 - inter_rect_y1 + 1, 0)
    inter_area = inter_area1*inter_area2
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

    return iou

def evaluateA(predictions):

    tFile = "HW/annotations.pickle"

    if not osp.exists(tFile):
        print("Error: The file " + tFile + "does not exist")
        return -1

    file = open(tFile,"rb")
    targets = pickle.load(file)

    imgCnt = len(targets.keys())

    # Variables for task 1
    nObj = 0
    nCorr = 0
    nPred = 0

    # Variables for task 1 HC
    nVAndC = 0
    nCorrVandC = 0

    # Variables for task 2
    nTS = 0
    nCorrTS = 0

    # Variables for task 2 HC
    nTSExtra = 0
    nCorrTSExtra = 0

    # Variables for task 3
    coordError = 0

    # Variables for task 3 HC
    poseError = 0

    tabla_meret =0
    veh_meret = 0
    cac_meret = 0
    tabla_oszt = 0
    veh_oszt =0
    cac_oszt = 0

    tabla_w, veh_w, cac_w, tabla_s, veh_s, cac_s = 0,0,0,0,0,0

    for k in targets.keys():

        target = targets[k]

        if k not in predictions.keys():
            print("Error: The image " + k + " does not exist in the predictions")
            return -1

        prediction = predictions[k]

        if 'poses' not in prediction.keys() or 'objects' not in prediction.keys():
            print("Error: The prediction for an image has to contain keys 'poses' and 'objects'")
            return -1

        tPose = target['poses']
        pPose = prediction['poses']

        tObjects = np.array(target['objects'])
        pObjects = np.array(prediction['objects'])

        '''bbNoise = np.random.randn(pObjects.shape[0],4)
        pObjects[:,:4] += bbNoise*0
        posNoise = np.random.randn(pObjects.shape[0],3)
        pObjects[:,6:] += posNoise*0
        poseNoise = np.random.randn(12)
        pPose += poseNoise*0'''

        # Task 3 HC: Reprojection error
        poseError += rbf(((tPose - pPose) ** 2).sum(), sigma=0.5)

        # Task 1: Recall, precision using IoU and main classes
        nObj += tObjects.shape[0]
        nPred += pObjects.shape[0]

        for obj in tObjects:

            # Compute IoUs
            IoUs = bbox_iou(obj,pObjects)

            # Get best box
            idx = np.argmax(IoUs)
            bestIoU = IoUs[idx]

            # If IoU and class are good
            if bestIoU > 0.25 and obj[4] == pObjects[idx,4]:

                # Correct predictions
                nCorr += 1

                # Euclidean coord distance
                coordError += rbf(((obj[6:] - pObjects[idx,6:]) ** 2).sum(), sigma=0.05)

                # Count correctly detected traffic signs
                if pObjects[idx,4] == 0:
                    if pObjects[idx, 5] < 52:
                        nTS += 1
                    else:
                        nTSExtra += 1

                # If secondary class is good
                if pObjects[idx, 5] == obj[5]:

                    if pObjects[idx, 4] == 0:
                        tabla_s += 1

                    if pObjects[idx, 4] == 1:
                        veh_s += 1
  
                    if pObjects[idx, 4] == 2:
                        cac_s += 1
  
                    # Increment counters based on the class
                    if obj[4] > 0:
                        nCorrVandC += 1
                    else:
                        if obj[5] < 52:
                            nCorrTS += 1
                        else:
                            nCorrTSExtra += 1
                else:
                  if pObjects[idx, 4] == 0:
                      tabla_w += 1

                  if pObjects[idx, 4] == 1:
                      veh_w += 1
  
                  if pObjects[idx, 4] == 2:
                      cac_w += 1
  

            elif bestIoU <= 0.25:
              if pObjects[idx, 4] == 0:
                  tabla_meret += 1
                  print("tabla_meret", k, pObjects[idx])
              if pObjects[idx, 4] == 1:
                  veh_meret += 1
                  print("veh_meret", k, pObjects[idx])
              if pObjects[idx, 4] == 2:
                  cac_meret += 1
                  print("cac_meret", k, pObjects[idx])

            else:
              if pObjects[idx, 4] == 0:
                  tabla_oszt += 1
                  print("tabla_oszt", k, pObjects[idx])
              if pObjects[idx, 4] == 1:
                  veh_oszt += 1
                  print("veh_oszt", k, pObjects[idx])
              if pObjects[idx, 4] == 2:
                  cac_oszt += 1
                  print("cac_oszt", k, pObjects[idx])

        # Task 1 HC: Classification accuracy of vehicle and cactus
        VNCObjects = np.array([obj for obj in tObjects if obj[4] > 0])
        nVAndC += VNCObjects.shape[0]

    print(tabla_meret,veh_meret,cac_meret, tabla_oszt,veh_oszt,cac_oszt)
    print(tabla_w, veh_w, cac_w, tabla_s, veh_s, cac_s)
    recall = nCorr/nObj if nObj else 1
    precision = nCorr/nPred if nPred else 1

    task1Score = (recall + precision)/2
    task1HCScore = nCorrVandC/nVAndC

    task2Score = nCorrTS/nTS if nTS else 0
    task2HCScore = nCorrTSExtra/nTSExtra if nTSExtra else 0

    task3Score = coordError/nCorr
    task3HCScore = poseError/imgCnt

    print("Task 1:", task1Score)
    print("Task 1 HC:", task1HCScore)

    print("Task 2:", task2Score)
    print("Task 2 HC:", task2HCScore)

    print("Task 3:", task3Score)
    print("Task 3 HC:", task3HCScore)

    print("Total: ", task1Score+task1HCScore+task2HCScore+task2Score+task3HCScore+task3Score)

In [0]:
#from HW.evaluate import evaluate

with open('HW/myAnswers.pickle', 'wb') as handle:
  pickle.dump(myAnswers, handle, protocol=pickle.HIGHEST_PROTOCOL)
file = open('HW/myAnswers.pickle','rb')
predictions = pickle.load(file)
evaluateA(predictions)